In [156]:
import pandas as pd

In [157]:
ml_m = pd.read_csv("./data/ml-100k/movies.csv")
ml_r = pd.read_csv("./data/ml-100k/ratings.csv")
ml_g = pd.read_csv("./data/ml-100k/movie_genres.csv")

In [158]:
# TODO: MovieLens single KG
'''
User: userId
Item: movieId & title
Attributes: genres, release_year
Relation: rating
'''
ml_kg = pd.merge(ml_r, ml_m, on=["movieId"], how="inner")

In [159]:
ml_kg.drop(labels=["timestamp", "release_date", "video_release_date", "IMDb_URL"], axis=1, inplace=True)

In [160]:
ml_kg

,rating_id,userId,movieId,rating,title
0,1,196,242,3,Kolya (1996)
1,254,63,242,3,Kolya (1996)
2,630,226,242,5,Kolya (1996)
3,1233,154,242,3,Kolya (1996)
4,2160,306,242,5,Kolya (1996)
...,...,...,...,...,...
99995,96445,840,1674,4,Mamma Roma (1962)
99996,97650,655,1640,3,"Eighth Day, The (1996)"
99997,98428,655,1637,3,Girls Town (1996)
99998,98641,655,1630,3,"Silence of the Palace, The (Saimt el Qusur) (1..."


In [161]:
ml_kg = pd.merge(ml_kg, ml_g, on=["movieId"], how="inner")

In [162]:
ml_kg.userId.nunique()

943

In [163]:
ml_kg.movieId.nunique()

1682

In [164]:
ml_kg = ml_kg[~ml_kg["title"].str.contains(r"1975-1979", regex=True)]

In [165]:
ml_kg = ml_kg[~ml_kg["title"].str.contains(r"2007-", regex=True)]

In [166]:
ml_kg[ml_kg["title"].str.contains(r"Frankenstein$", regex=True)]

,rating_id,userId,movieId,rating,title,genreId,genres


In [167]:
def get_year(x:str):
    try:
        return str(int(x.split()[-1].strip("\(").strip("\)")) // 10 * 10) + "\'s"
    except:
        return ""

ml_kg["release_year"] = ml_kg["title"].apply(lambda x:get_year(x))

In [168]:
def p_title(x:str):
    try:
        return " ".join(x.split()[:-1])
    except:
        return x

ml_kg["title"] = ml_kg["title"].apply(lambda x:p_title(x))

In [169]:
ml_kg

,rating_id,userId,movieId,rating,title,genreId,genres,release_year
0,1,196,242,3,Kolya,633,Comedy,1990's
1,1,196,242,3,Kolya,3526,Comedy,1990's
2,254,63,242,3,Kolya,633,Comedy,1990's
3,254,63,242,3,Kolya,3526,Comedy,1990's
4,630,226,242,5,Kolya,633,Comedy,1990's
...,...,...,...,...,...,...,...,...
425185,98428,655,1637,3,Girls Town,4803,Drama,1990's
425186,98641,655,1630,3,"Silence of the Palace, The (Saimt el Qusur)",1904,Drama,1990's
425187,98641,655,1630,3,"Silence of the Palace, The (Saimt el Qusur)",4797,Drama,1990's
425188,98956,655,1641,3,Dadetown,1215,Documentary,1990's


In [170]:
ml_kg.iloc[0, :]["movieId"]

242

In [171]:
from collections import defaultdict

m_g_dic = defaultdict(list)
for i in range(len(ml_kg)):
    m_g_dic[ml_kg.iloc[i, :]["movieId"]].append(ml_kg.iloc[i, :]["genres"])

In [172]:
for k, v in m_g_dic.items():
    m_g_dic[k] = list(set(v))

In [173]:
ml_kg["genre_list"] = ml_kg["movieId"].map(m_g_dic)

In [176]:
ml_kg_ = ml_kg[:]

In [177]:
def f(x:list):
    x = sorted(x)
    return "|".join(x)

ml_kg_["genre_list"] = ml_kg["genre_list"].apply(lambda x:f(x))

In [178]:
ml_kg_ = ml_kg_.drop(columns=["genres"]).drop(columns=["genreId"]).drop_duplicates()

In [179]:
ml_kg_

,rating_id,userId,movieId,rating,title,release_year,genre_list
0,1,196,242,3,Kolya,1990's,Comedy
2,254,63,242,3,Kolya,1990's,Comedy
4,630,226,242,5,Kolya,1990's,Comedy
6,1233,154,242,3,Kolya,1990's,Comedy
8,2160,306,242,5,Kolya,1990's,Comedy
...,...,...,...,...,...,...,...
425180,96445,840,1674,4,Mamma Roma,1960's,Drama
425182,97650,655,1640,3,"Eighth Day, The",1990's,Drama
425184,98428,655,1637,3,Girls Town,1990's,Drama
425186,98641,655,1630,3,"Silence of the Palace, The (Saimt el Qusur)",1990's,Drama


In [181]:
ml_tmdb = pd.read_csv("./data/ml_tmdb.csv")

In [182]:
ml_tmdb.drop(labels=["movieId", "release_year"], inplace=True, axis=1)

In [183]:
len(set(ml_tmdb.title.unique()) & set(ml_kg.title.unique()))

1122

In [195]:
ml_kg_.drop(labels=["rating_id"], axis=1, inplace=True)

In [196]:
df = pd.merge(ml_kg_, ml_tmdb, on=["title"], how="left")

In [197]:
df = df.drop_duplicates(subset=["userId", "movieId"])

In [198]:
df[df["origin_country"] == "Null"].loc[:, "origin_country"] = df[df["origin_country"] == "Null"]["origin_country"].apply(lambda x:"")

In [199]:
df = df.fillna("")

In [200]:
def f5(x):
    if x == "Null":
        return ""
    else:
        return x

df["origin_country"] = df["origin_country"].apply(lambda x:f5(x))

In [201]:
df.to_csv("./data/ml-100k_tmdb_merged.csv", index=True)

In [202]:
df["genre_list"]

0              Comedy
1              Comedy
2              Comedy
3              Comedy
4              Comedy
             ...     
111377          Drama
111378          Drama
111379          Drama
111380          Drama
111381    Documentary
Name: genre_list, Length: 100000, dtype: object

In [204]:
df

,userId,movieId,rating,title,release_year,genre_list,origin_country,writer,director,cast
0,196,242,3,Kolya,1990's,Comedy,CZ,,Jan Svěrák,Silvia Šuvadová
1,63,242,3,Kolya,1990's,Comedy,CZ,,Jan Svěrák,Silvia Šuvadová
2,226,242,5,Kolya,1990's,Comedy,CZ,,Jan Svěrák,Silvia Šuvadová
3,154,242,3,Kolya,1990's,Comedy,CZ,,Jan Svěrák,Silvia Šuvadová
4,306,242,5,Kolya,1990's,Comedy,CZ,,Jan Svěrák,Silvia Šuvadová
...,...,...,...,...,...,...,...,...,...,...
111377,840,1674,4,Mamma Roma,1960's,Drama,IT,,Pier Paolo Pasolini,Anna Magnani
111378,655,1640,3,"Eighth Day, The",1990's,Drama,,,,
111379,655,1637,3,Girls Town,1990's,Drama,,Anna Grace,Jim McKay,Lili Taylor
111380,655,1630,3,"Silence of the Palace, The (Saimt el Qusur)",1990's,Drama,,,,


In [205]:
df.to_csv("./data/ml-100k_tmdb_final.csv", index=False)